# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather_data = pd.read_csv('cities.csv')
cities_weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,32.56,97,67,2.30,NZ,1659112987
1,1,new norfolk,-42.7826,147.0587,42.30,84,99,2.57,AU,1659112820
2,2,barra do garcas,-15.8900,-52.2567,90.18,29,0,11.50,BR,1659112987
3,3,hobart,-42.8794,147.3294,42.66,82,96,6.91,AU,1659112988
4,4,bako,5.7833,36.5667,67.59,70,90,3.00,ET,1659112988
...,...,...,...,...,...,...,...,...,...,...
549,549,kalmunai,7.4167,81.8167,80.08,82,100,10.94,LK,1659113139
550,550,quixeramobim,-5.1992,-39.2928,95.32,30,49,7.36,BR,1659113139
551,551,faya,18.3851,42.4509,71.71,56,93,6.82,SA,1659113140
552,552,gornopravdinsk,60.0500,69.9000,64.76,70,21,4.27,RU,1659113140


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_weather_data[["Lat", "Lng"]]
humidity = cities_weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.


In [5]:
cities_pleasant_temp = cities_weather_data[cities_weather_data['Max Temp'].between(70,80)]                                 
cities_pleasant_temp

cities_pleasant_wind = cities_pleasant_temp[cities_pleasant_temp['Wind Speed'] < 10]   
cities_pleasant_wind

hotel_df = cities_pleasant_wind[cities_pleasant_wind['Cloudiness'] == 0]  

hotel_df['Hotel Name'] = ""


hotel_df


C:\Users\pravi\AppData\Local\Temp\ipykernel_22256\817357195.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,37,ponta do sol,32.6667,-17.1000,76.12,64,0,3.60,PT,1659112997,
51,51,hilo,19.7297,-155.0900,77.29,71,0,8.99,US,1659112740,
85,85,zhicheng,30.2956,111.5047,77.18,83,0,3.69,CN,1659113010,
162,162,linfen,36.0889,111.5189,78.62,67,0,3.09,CN,1659113031,
184,184,nanyang,32.9947,112.5328,74.48,83,0,5.95,CN,1659113037,
194,194,lagoa,39.0500,-27.9833,72.48,72,0,1.77,PT,1659113040,
217,217,urumqi,43.8010,87.6005,79.75,34,0,8.95,CN,1659113050,
222,222,slave lake,55.2834,-114.7690,71.58,73,0,3.44,CA,1659113051,
350,350,octeville,49.6261,-1.6435,73.89,49,0,4.61,FR,1659113086,
358,358,iskilip,40.7353,34.4739,74.98,37,0,8.61,TR,1659113088,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# find the closest restaurant of each type to coordinates
distance = 5000

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": distance,
    "types": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # get restaurant type from df
    lat = row['Lat']
    lng = row['Lng']
    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {lodging}.")
    response = requests.get(base_url, params=params).json()
      
    try:
                 
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------


In [7]:
hotel_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,37,ponta do sol,32.6667,-17.1000,76.12,64,0,3.60,PT,1659112997,Hotel do Campo
51,51,hilo,19.7297,-155.0900,77.29,71,0,8.99,US,1659112740,Hilo Hawaiian Hotel
85,85,zhicheng,30.2956,111.5047,77.18,83,0,3.69,CN,1659113010,Wangjiang Building
162,162,linfen,36.0889,111.5189,78.62,67,0,3.09,CN,1659113031,Suoxing Hotel
184,184,nanyang,32.9947,112.5328,74.48,83,0,5.95,CN,1659113037,Wolong Memory Hotel
194,194,lagoa,39.0500,-27.9833,72.48,72,0,1.77,PT,1659113040,Casa Das Faias
217,217,urumqi,43.8010,87.6005,79.75,34,0,8.95,CN,1659113050,Hua Ling Grand Hotel Urumqi
222,222,slave lake,55.2834,-114.7690,71.58,73,0,3.44,CA,1659113051,Super 8 by Wyndham Slave Lake AB
350,350,octeville,49.6261,-1.6435,73.89,49,0,4.61,FR,1659113086,Hôtel Le Louvre Cherbourg
358,358,iskilip,40.7353,34.4739,74.98,37,0,8.61,TR,1659113088,İskilip Öğretmenevi


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]



In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info)

# Plot Heatmap
fig = gmaps.figure()


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))